In [1]:
# Imports

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error # For comparisson between lines plotted, usage up for discussion.

### Import and Optimiza Data

UN WPP has 2 different sets of data listed on their CSV downloads, which can be found if the `./data/` path.For this project the `./data/data_Medium.csv` is used to derive analytics and visualizations. Below, you can find the code related to import and optimization of dtypes for said data file.  

In [26]:
# Read data from .parquet file
df = pd.read_parquet('./data/data_Medium.parquet')

# Inspect dtypes
# df.info(memory_usage='deep')

# Check if reserved bits can be reduced for any columns of type int64

int_cols = df.select_dtypes(include='int64')
for col in int_cols.columns:
    
    if (df[col].max() < 32767):
        df[col] = df[col].astype('int16')

# Set 'Time' as type datetime64[ns]
df['Time'] = pd.to_datetime(df['Time'], format='%Y') # format = '%Y' is the only matching format for the dataset.

# df.info(memory_usage='deep') for comparrison